In [6]:
import pandas as pd
import numpy as np
import warnings
from glob import glob

# Càrrega i neteja de les dades

In [18]:
warnings.filterwarnings(action='ignore', category=FutureWarning, 
                        message=r".*Use a DatetimeIndex.*")

warnings.filterwarnings(action='ignore', category=UserWarning)
pd.options.mode.copy_on_write = True

In [ ]:
path_iris = 'data/comunicacions'
cat_vars = ['TIPUS', 'AREA', 'ELEMENT', 'DETALL', 'DISTRICTE', 'BARRI']
lst_dfs = []

for file in glob('/'.join([path_iris, '*.csv'])):
    df = pd.read_csv(file, parse_dates=[["DIA_DATA_ALTA", "MES_DATA_ALTA", "ANY_DATA_ALTA"]], date_format="mixed", dayfirst=True)
    df['DATA_ALTA'] = df['DIA_DATA_ALTA_MES_DATA_ALTA_ANY_DATA_ALTA']
    
    for col in cat_vars:
        df[col] = df[col].str.replace('""', '')
    
    lst_dfs.append(df)

data_iris = pd.concat(lst_dfs, axis=0)
data_iris = data_iris[["FITXA_ID","TIPUS","AREA","ELEMENT","DETALL",
                       'DATA_ALTA',
                       "DISTRICTE","BARRI"]]

data_iris.info()

C:\Users\JM_Data_Science\AppData\Local\Temp\ipykernel_17612\1965815548.py:6: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(file, parse_dates=[["DIA_DATA_ALTA", "MES_DATA_ALTA", "ANY_DATA_ALTA"]], date_format="mixed", dayfirst=True)
C:\Users\JM_Data_Science\AppData\Local\Temp\ipykernel_17612\1965815548.py:6: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(file, parse_dates=[["DIA_DATA_ALTA", "MES_DATA_ALTA", "ANY_DATA_ALTA"]], date_format="mixed", dayfirst=True)
C:\Users\JM_Data_Science\AppData\Local\Temp\ipykernel_17612\1965815548.py:6: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, parse_dates=[["DIA_DATA_ALTA", "MES_DATA_ALTA", "ANY_DATA_ALT

<class 'pandas.core.frame.DataFrame'>
Index: 496698 entries, 0 to 249241
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   FITXA_ID   496698 non-null  int64         
 1   TIPUS      496698 non-null  object        
 2   AREA       496698 non-null  object        
 3   ELEMENT    496698 non-null  object        
 4   DETALL     496698 non-null  object        
 5   DATA_ALTA  496698 non-null  datetime64[ns]
 6   DISTRICTE  319530 non-null  object        
 7   BARRI      319075 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 34.1+ MB


In [50]:
districtes = ['Ciutat Vella', 'Eixample', 'Sants-Montjuïc', 'Les Corts', 'Sarrià-Sant Gervasi', 'Gràcia', 'Horta-Guinardó', 'Nou Barris', 'Sant Andreu', 'Sant Martí']
mask_districtes = data_iris.DISTRICTE.isin(districtes)
data_iris = data_iris[mask_districtes]

data_iris = data_iris.dropna(axis=0, subset=['DISTRICTE', 'BARRI', 'ELEMENT'])

buit_mask = data_iris.BARRI == '"'
data_iris = data_iris[buit_mask == False]

n_iris_clean = len(data_iris)

classes_neteja = ["Recollida animals morts", "Recollida comercial", 
                  "Recollida de mobles i trastos vells", "Recollida de poda al domicili", 
                  "Recollida i neteja de l’espai urbà", "Recollida orgànic / rebuig",
                  "Recollida paper cartró/vidre/reciclables",
                  "Recollida pneumàtica", "Recollida roba domiciliària", "Recollida selectiva mòbil",
                  "Recollida selectiva porta a porta", "Cont. intel.residus orgànics", "Neteja",
                  "Neteja carrers i/o places", "Papereres", "Implantació de la nova contracta neteja",
                  "Lavabo públic", "Punts verds", "Neteja solar municipal", "Platges neteja"]
mask_neteja = data_iris.ELEMENT.isin(classes_neteja)

data_neteja = data_iris[mask_neteja]

n_neteja = len(data_neteja)

print(f"Una vegada realitzada la neteja de les dades, disposem de {n_iris_clean} comunicacions, de les que {n_neteja} són referents a temàtiques relacionadas amb la neteja municipal.")


Una vegada realitzada la neteja de les dades, disposem de 318251 comunicacions, de les que 128860 són referents a temàtiques relacionadas amb la neteja municipal.


In [51]:
data_neteja.head()

,FITXA_ID,TIPUS,AREA,ELEMENT,DETALL,DATA_ALTA,DISTRICTE,BARRI
0,23314205,INCIDENCIA,Recollida i neteja de l'espai urbà,Neteja carrers i/o places,Objectes a netejar / retirar,2022-12-22,Horta-Guinardó,Horta
1,23314208,INCIDENCIA,Recollida i neteja de l'espai urbà,Neteja carrers i/o places,Objectes a netejar / retirar,2022-12-22,Horta-Guinardó,Horta
3,23314206,INCIDENCIA,Recollida i neteja de l'espai urbà,Neteja carrers i/o places,Objectes a netejar / retirar,2022-12-22,Horta-Guinardó,la Teixonera
58,23329218,INCIDENCIA,Recollida i neteja de l'espai urbà,Neteja carrers i/o places,Objectes a netejar / retirar,2023-01-01,Eixample,la Sagrada Família
60,23329217,INCIDENCIA,Recollida i neteja de l'espai urbà,Neteja carrers i/o places,Objectes a netejar / retirar,2023-01-01,Eixample,l'Antiga Esquerra de l'Eixample


Metadades on s'informa de la codificació per gènere 1 = Dona i 2 = Home:

https://opendata-ajuntament.barcelona.cat/data/ca/dataset/pad-dimensions/resource/b00be3f8-9328-4175-8689-24a25bc0907c


In [104]:
path_demografia = 'data/demografia'
lst_dfs = []

for file in glob('/'.join([path_demografia, '*.csv'])):
    df = pd.read_csv(file)
    lst_dfs.append(df)

data_demografia = pd.concat(lst_dfs)

data_demografia.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4272 entries, 0 to 2135
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Data_Referencia  4272 non-null   object
 1   Codi_Districte   4272 non-null   int64 
 2   Nom_Districte    4272 non-null   object
 3   Codi_Barri       4272 non-null   int64 
 4   Nom_Barri        4272 non-null   object
 5   AEB              4272 non-null   int64 
 6   Seccio_Censal    4272 non-null   int64 
 7   Valor            4272 non-null   int64 
 8   SEXE             4272 non-null   int64 
dtypes: int64(6), object(3)
memory usage: 333.8+ KB


In [105]:
data_demografia['Any_Referencia'] = data_demografia.Data_Referencia.str.split("-", n=1, expand=True)[0]

mask_Homes = data_demografia.SEXE == 2
data_demografia['SEXE_DESCODIF'] = "Dones"
data_demografia.loc[mask_Homes, 'SEXE_DESCODIF'] = "Homes"

data_demografia = data_demografia[['Any_Referencia', 'Nom_Districte', 'Nom_Barri', 'Valor', 'SEXE_DESCODIF']]

In [106]:
data_demografia.head()

,Any_Referencia,Nom_Districte,Nom_Barri,Valor,SEXE_DESCODIF
0,2023,Ciutat Vella,el Raval,606,Dones
1,2023,Ciutat Vella,el Raval,640,Homes
2,2023,Ciutat Vella,el Raval,588,Dones
3,2023,Ciutat Vella,el Raval,650,Homes
4,2023,Ciutat Vella,el Raval,1555,Dones


In [55]:
data_superficie = pd.read_csv('data/2021_superficie.csv')

data_superficie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Any              73 non-null     int64  
 1   Codi_Districte   73 non-null     int64  
 2   Nom_Districte    73 non-null     object 
 3   Codi_Barri       73 non-null     int64  
 4   Nom_Barri        73 non-null     object 
 5   Superfície (ha)  73 non-null     float64
dtypes: float64(1), int64(3), object(2)
memory usage: 3.6+ KB


In [121]:
data_superficie = data_superficie[['Nom_Districte', 'Nom_Barri', 'Superfície (ha)']]
data_superficie.columns = ['Nom_Districte', 'Nom_Barri', 'Superficie']

In [122]:
data_superficie.head()

,Nom_Districte,Nom_Barri,Superficie
0,Ciutat Vella,el Raval,110.0
1,Ciutat Vella,el Barri Gòtic,81.6
2,Ciutat Vella,la Barceloneta,117.9
3,Ciutat Vella,"Sant Pere, Santa Caterina i la Ribera",111.0
4,Eixample,el Fort Pienc,92.9


In [137]:
data_prop_usos = pd.read_csv("data/Taula_prop_usos_sol.csv")

data_prop_usos.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Territori           252 non-null    object
 1   Tipus de territori  252 non-null    object
 2   Ús                  252 non-null    object
 3   2010                252 non-null    object
 4   2011                252 non-null    object
 5   2012                252 non-null    object
 6   2013                252 non-null    object
 7   2014                252 non-null    object
 8   2015                252 non-null    object
 9   2016                252 non-null    object
 10  2017                252 non-null    object
 11  2018                252 non-null    object
 12  2019                252 non-null    object
 13  2020                252 non-null    object
 14  2021                252 non-null    object
 15  2022                252 non-null    object
dtypes: object(16)
memory usage

In [138]:
mask_residencial = data_prop_usos['Ús'] == "Residencial"
data_prop_residencial = data_prop_usos[mask_residencial]

mask_dash = data_prop_residencial['2022'] == "-"
data_prop_residencial[['Territori', '2022']][mask_dash == False]


,Territori,2022
0,Barcelona,25.47
3,Ciutat Vella,30.84
18,Eixample,49.83
39,Sants-Montjuïc,10.93
66,Les Corts,29.66
78,Sarrià-Sant Gervasi,22.72
99,Gràcia,48.06
117,Horta-Guinardó,24.31
153,Nou Barris,28.63
195,Sant Andreu (Districte),28.96


Només prendrem en consideració les proporcions calculades per l'any 2021 atès que aquestes sí les disposem de forma completa per tots els barris.

In [139]:
data_prop_residencial = data_prop_residencial[['Territori', 'Tipus de territori', 'Ús', '2021']]

data_prop_residencial.head()

,Territori,Tipus de territori,Ús,2021
0,Barcelona,Municipi,Residencial,27.09
3,Ciutat Vella,Districte,Residencial,31.46
6,el Raval,Barri,Residencial,45.07
9,el Barri Gòtic,Barri,Residencial,41.85
12,la Barceloneta,Barri,Residencial,12.36


In [81]:
data_precipitacions = pd.read_csv('data/precipitacionsbcndesde1786_2024_long.csv')

data_precipitacions.tail()

,Any,Mes,Desc_Mes,Precipitacions
2863,2024,8,Agost,17.6
2864,2024,9,Setembre,70.7
2865,2024,10,Octubre,118.5
2866,2024,11,Novembre,54.3
2867,2024,12,Desembre,38.4


In [82]:
data_temperatura = pd.read_csv('data/temperaturesbcndesde1780_2024_long.csv')

data_temperatura.tail()

,Any,Mes,Desc_Mes,Temperatura
2935,2024,8,Agost,26.9
2936,2024,9,Setembre,20.8
2937,2024,10,Octubre,18.2
2938,2024,11,Novembre,14.8
2939,2024,12,Desembre,10.4


# Transformacions i agregacions

In [88]:
data_neteja['ANY_ALTA'] = data_neteja.DATA_ALTA.dt.year.astype(int)

data_neteja['MES_ALTA_NOMBRE'] = data_neteja.DATA_ALTA.dt.month

mesos = [{'nom_mes': 'Gener', 'nombre_mes': 1}, {'nom_mes': 'Febrer', 'nombre_mes': 2},
         {'nom_mes': 'Març', 'nombre_mes': 3}, {'nom_mes': 'Abril', 'nombre_mes': 4},
         {'nom_mes': 'Maig', 'nombre_mes': 5}, {'nom_mes': 'Juny', 'nombre_mes': 6},
         {'nom_mes': 'Juliol', 'nombre_mes': 7}, {'nom_mes': 'Agost', 'nombre_mes': 8},
         {'nom_mes': 'Setembre', 'nombre_mes': 9}, {'nom_mes': 'Octubre', 'nombre_mes': 10},
         {'nom_mes': 'Novembre', 'nombre_mes': 11}, {'nom_mes': 'Desembre', 'nombre_mes': 12}]

data_neteja['MES_ALTA_NOM'] = ''

for mes in mesos:
    nombre_mes = mes['nombre_mes']
    nom_mes = mes['nom_mes']
    mask = data_neteja.MES_ALTA_NOMBRE == nombre_mes
    data_neteja.loc[mask, 'MES_ALTA_NOM'] = nom_mes

data_neteja['TRIMESTRE_ALTA'] = data_neteja.DATA_ALTA.dt.quarter

mask_2022 = data_neteja.ANY_ALTA == 2022

data_neteja = data_neteja[mask_2022 == False].reset_index(drop=True)

data_neteja.head()

,FITXA_ID,TIPUS,AREA,ELEMENT,DETALL,DATA_ALTA,DISTRICTE,BARRI,ANY_ALTA,MES_ALTA_NOMBRE,MES_ALTA_NOM,TRIMESTRE_ALTA
0,23329218,INCIDENCIA,Recollida i neteja de l'espai urbà,Neteja carrers i/o places,Objectes a netejar / retirar,2023-01-01,Eixample,la Sagrada Família,2023,1,Gener,1
1,23329217,INCIDENCIA,Recollida i neteja de l'espai urbà,Neteja carrers i/o places,Objectes a netejar / retirar,2023-01-01,Eixample,l'Antiga Esquerra de l'Eixample,2023,1,Gener,1
2,23329221,INCIDENCIA,Recollida i neteja de l'espai urbà,Neteja carrers i/o places,Objectes a netejar / retirar,2023-01-01,Sarrià-Sant Gervasi,Sant Gervasi - la Bonanova,2023,1,Gener,1
3,23329211,INCIDENCIA,Recollida i neteja de l'espai urbà,Neteja carrers i/o places,Objectes a netejar / retirar,2023-01-01,Horta-Guinardó,Can Baró,2023,1,Gener,1
4,23329212,INCIDENCIA,Recollida i neteja de l'espai urbà,Neteja carrers i/o places,Objectes a netejar / retirar,2023-01-01,Horta-Guinardó,el Guinardó,2023,1,Gener,1


In [93]:
groupby_neteja = data_neteja.groupby(['ELEMENT', 'DISTRICTE', 'BARRI', 'ANY_ALTA', 'MES_ALTA_NOM']).size().reset_index(drop=False)
groupby_neteja.columns = ['ELEMENT', 'DISTRICTE', 'BARRI', 'ANY_ALTA', 'MES_ALTA_NOM', 'RECOMPTE']

groupby_neteja.head()

,ELEMENT,DISTRICTE,BARRI,ANY_ALTA,MES_ALTA_NOM,RECOMPTE
0,Lavabo públic,Ciutat Vella,Sant Pere Santa Caterina i la Ribera,2023,Agost,1
1,Lavabo públic,Ciutat Vella,Sant Pere Santa Caterina i la Ribera,2023,Febrer,1
2,Lavabo públic,Ciutat Vella,Sant Pere Santa Caterina i la Ribera,2023,Juliol,5
3,Lavabo públic,Ciutat Vella,Sant Pere Santa Caterina i la Ribera,2023,Juny,1
4,Lavabo públic,Ciutat Vella,Sant Pere Santa Caterina i la Ribera,2023,Setembre,1


In [117]:
groupby_demografia = data_demografia.groupby(['Any_Referencia', 'Nom_Districte', 'Nom_Barri', 'SEXE_DESCODIF']).sum().reset_index(drop=False)
groupby_demografia.columns = ['Any_Referencia', 'Nom_Districte', 'Nom_Barri', 'TIPUS_AGREGACIO', 'Valor']
groupby_demografia_totals = groupby_demografia.groupby(['Any_Referencia', 'Nom_Districte', 'Nom_Barri']).sum().reset_index(drop=False)
groupby_demografia_totals.TIPUS_AGREGACIO = "Total"
groupby_demografia_tot = pd.concat([groupby_demografia_totals, groupby_demografia], axis=0)

df_wide = groupby_demografia_tot.pivot(index=['Any_Referencia', 'Nom_Districte', 'Nom_Barri'],
                   columns='TIPUS_AGREGACIO',
                   values='Valor').reset_index()

cols_order = ['Any_Referencia', 'Nom_Districte', 'Nom_Barri', 'Homes', 'Dones', 'Total']
df_wide = df_wide[cols_order]
df_wide.head()


TIPUS_AGREGACIO,Any_Referencia,Nom_Districte,Nom_Barri,Homes,Dones,Total
0,2023,Ciutat Vella,"Sant Pere, Santa Caterina i la Ribera",10868,11173,22041
1,2023,Ciutat Vella,el Barri Gòtic,15184,9276,24460
2,2023,Ciutat Vella,el Raval,24042,21629,45671
3,2023,Ciutat Vella,la Barceloneta,7042,7232,14274
4,2023,Eixample,Sant Antoni,18225,20044,38269


In [140]:
data_superficie_residencial = pd.merge(data_superficie, data_prop_residencial, how='left', left_on='Nom_Barri', right_on='Territori')
data_superficie_residencial['Superficie_Residencial'] = (data_superficie_residencial['Superficie'].astype(float) * data_superficie_residencial['2021'].astype(float) / 100.0).round(3)
data_superficie_residencial = data_superficie_residencial[['Nom_Districte', 'Nom_Barri', 'Superficie', 'Superficie_Residencial']]

data_superficie_residencial.head()


,Nom_Districte,Nom_Barri,Superficie,Superficie_Residencial
0,Ciutat Vella,el Raval,110.0,49.577
1,Ciutat Vella,el Barri Gòtic,81.6,34.150
2,Ciutat Vella,la Barceloneta,117.9,14.572
3,Ciutat Vella,"Sant Pere, Santa Caterina i la Ribera",111.0,32.401
4,Eixample,el Fort Pienc,92.9,33.621
